In [29]:
import os
import numpy as np
import random
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
from dataset import dataset
from neural_network import neural_net
from additional_functions import process_all
from additional_functions import save_arrays_to_csv
from additional_functions import load_arrays_from_csv
from additional_functions import make_plots_2

In [30]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
from dataset import dataset

class neural_net:
    def __init__(self, data: dataset, prediction_type_flag: str, hidden_layer_count=0, network_shape=[], hidden_node_count=1, epochs=100, momentum=.9, learning_rate=.01, batch_size=10, suppress_plots=True):
        self.suppress_plots = suppress_plots
        self.epochs = epochs
        self.momentum = momentum
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_layer_count = hidden_layer_count
        if hidden_layer_count == 0:
            hidden_node_count = 0
        else:
            hidden_node_count = [hidden_node_count] * hidden_layer_count
        self.tune_set = data.tune_set
        self.validate_set = data.validate_set
        self.prediction_type = prediction_type_flag

        if self.prediction_type == "classification":
            self.class_count = len(np.unique(self.tune_set[:,-1]))
        else:
            self.class_count = 0

        input_size = self.tune_set.shape[1] - 1
        self.input_size = input_size
        if network_shape == []:
            self.network_shape = [input_size] + (hidden_node_count if hidden_node_count else []) + ([self.class_count] if (self.prediction_type == "classification") else [1])
        else:
            self.network_shape = network_shape
        self.biases = []
        self.weights = []
        self.bias_velocity = []
        self.weight_velocity = []

    def init_weights_biases_momentum(self):
        '''
        Initializes weights based on the network shape list
        '''
        self.biases = [np.random.randn(next_size, 1) for next_size in self.network_shape[1:]]
        self.weights = [np.random.randn(next_size, cur_size) for cur_size, next_size in zip(self.network_shape[:-1], self.network_shape[1:])]
        self.bias_velocity = [np.zeros(bias.shape) for bias in self.biases]
        self.weight_velocity = [np.zeros(weight.shape) for weight in self.weights]
        #print(self.biases)


    def for_prop(self, input: np, testing=False):
        '''
        Feeds forward a single example through the network
        '''
        output = input
        for idx, (bias, weight) in enumerate(zip(self.biases[:-1], self.weights[:-1])):
            output = self.sigmoid(np.dot(weight, output) + bias)
            if testing:
                print(f"Activation at Layer {idx + 1}:\n{output}\n")
        # The output layer
        bias, weight = self.biases[-1], self.weights[-1]
        output = (np.dot(weight, output) + bias)
        if self.prediction_type == "classification":
            output = self.softmax(output)
        if testing:
            print(f"Activation at Output Layer:\n{output}\n")
        return output
    
    def get_training_data(self, i: int):
        '''
        method needs to take the set of fold i-(i-1) and and compile those into its own array.
        Then format the data as follows: each example = (attributes, label)
        i is used to indicate which training set you want returned
        '''
        desired_data = np.concatenate([self.validate_set[j] for j in range(10) if j != i])
        training_data = [(example[:-1], example[-1]) for example in desired_data]
        return training_data
    
    def get_testing_data(self, i: int):
        '''
        method needs to take the set of fold i-(i-1) and and compile those into its own array.
        Then format the data as follows: each example = (attributes, label)
        i is used to indicate which training set you want returned
        '''
        desired_data = self.validate_set[i]
        testing_data = [(example[:-1], example[-1]) for example in desired_data]
        return testing_data
    
    def get_tuning_data(self):
        '''
        method needs to take the set of fold i-(i-1) and and compile those into its own array.
        Then format the data as follows: each example = (attributes, label)
        i is used to indicate which training set you want returned
        '''
        desired_data = self.tune_set
        tuning_data = [(example[:-1], example[-1]) for example in desired_data]
        return tuning_data

    def grad_desc(self, training_data, epochs, momentum, learning_rate, batch_size):
        '''
        Takes in a traing set from get_training_data. The format is a list of tuples, where each tuple
        represents an example. Within each tuple the first value is the feature vector and the second
        value is the label.
        '''
        example_count = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+batch_size] for k in range(0, example_count, batch_size)]
            for mini_batch in mini_batches:
                self.update_weights(mini_batch, momentum, learning_rate)

    def update_weights(self, mini_batch, momentum, learning_rate, testing=False):
        '''
        Demonstrates weight updates for each layer of a two-layer network
        '''
        bias_gradient = [np.zeros(bias.shape) for bias in self.biases]
        weight_gradient = [np.zeros(weight.shape) for weight in self.weights]

        # Compute gradients for the mini-batch
        for feature, label in mini_batch:
            if not np.isnan(label):
                delta_bias_gradient, delta_weight_gradient = self.epoch(feature, label)
                bias_gradient = [gradient + delta for gradient, delta in zip(bias_gradient, delta_bias_gradient)]
                weight_gradient = [gradient + delta for gradient, delta in zip(weight_gradient, delta_weight_gradient)]
        
        # Update velocities and apply updates with momentum
        self.bias_velocity = [momentum * velocity - (learning_rate / len(mini_batch)) * gradient for velocity, gradient in zip(self.bias_velocity, bias_gradient)]
        self.weight_velocity = [momentum * velocity - (learning_rate / len(mini_batch)) * gradient for velocity, gradient in zip(self.weight_velocity, weight_gradient)]

        # Update weights and biases, and print the updates for each layer
        for idx, (bias, velocity) in enumerate(zip(self.biases, self.bias_velocity)):
            self.biases[idx] = bias + velocity
            #if testing:
                #print(f"Bias update at Layer {idx + 1}:{self.biases[idx]}")
        for idx, (weight, velocity) in enumerate(zip(self.weights, self.weight_velocity)):
            self.weights[idx] = weight + velocity
            if testing:
                print(f"Weight update at Layer {idx + 1}:{self.weights[idx]}")

    def epoch(self, feature, label, testing=False):
        '''
        NEEDS COMMENTING/PARSING
        '''
        #print(f"\n\nLABEL: {label}\n\n")
        bias_gradient = [np.zeros(bias.shape) for bias in self.biases]
        weight_gradient = [np.zeros(weight.shape) for weight in self.weights]
        # feedforward
        activation = feature
        activations = [feature] # list to store all the activations, layer by layer
        weighted_inputs = [] # list to store all the z vectors, layer by layer
        #print(f"Biases: {self.biases[-1]}")
        #print(f"Weights: {self.weights[-1]}")
        for bias, weight in zip(self.biases[:-1], self.weights[:-1]):
            #print(f"Activation:\n{activation.shape}\n\nWeight:\n{weight.shape}\n\nBias:\n{bias.shape}\n\n\n")
            weighted_input = np.dot(weight, activation.reshape(-1,1)) + bias
            #print(f"Weighted Input:\n{weighted_input.shape}")
            activation = self.sigmoid(weighted_input)
            weighted_inputs.append(weighted_input)
            activations.append(activation)
        # The output layer uses different activation functions
        bias, weight = self.biases[-1], self.weights[-1]
        #print(f"Activation:\n{activation}\n\nWeight:\n{weight.shape}\n\nBias:\n{bias.shape}\n\n\n")
        #print(f"Dot Product Shape:\n{np.dot(weight, activation).reshape(-1,1)}\n\n")
        weighted_input = np.dot(weight, activation.reshape(-1,1)) + bias
        #weighted_input = np.dot(weight, activation)
        #print(f"Weighted Input (Should be two scalars):\n{(weighted_input.shape)}")
        activation = weighted_input
        if self.prediction_type == "classification":
            activation = self.softmax(weighted_input)
        weighted_inputs.append(weighted_input)
        activations.append(activation)
        #print(f"Activations:\n{activations}\n\nWeighted Inputs:\n{weighted_inputs}")
    

        # Print gradients at the output layer
        if self.prediction_type == "classification":
            one_hot_label = [0] * self.class_count
            one_hot_label[int(label)] = 1
            one_hot_label = np.array(one_hot_label).reshape(-1, 1)
        else:
            one_hot_label = label
        #print(f"Activations[-1]:\n{activations[-1]}\nOne-Hot Label:\n{one_hot_label}")
        delta = (activations[-1] - one_hot_label)# * self.softmax(weighted_inputs[-1])
        if testing:
            print(f"Activations:\n{activations[-1]}")
            print(f"Label:\n{one_hot_label}")
            print()
            print(f"Gradient at Output Layer (delta):\n{delta}\n")
        bias_gradient[-1] = delta
        #print(f"Delta:\n{delta}\n\nActivations:\n{activations[-2].reshape(1,-1)}\n\n")
        weight_gradient[-1] = np.dot(delta, activations[-2].reshape(1,-1))# # CHECK THIS LINE FOR COMPREHENSION

        for layer_idx in range(2, len(self.network_shape)):
            weighted_input = weighted_inputs[-layer_idx]
            activation_prime = self.sigmoid_prime(weighted_input)
            delta = np.dot(self.weights[-layer_idx+1].transpose(), delta) * activation_prime
            # add logic to convert scalar if delta is 1x1
            if delta.shape == (1,1):
                bias_gradient[-layer_idx] = delta.reshape(-1)
                weight_gradient[-layer_idx] = (delta.reshape(-1) * activations[-layer_idx-1].transpose())
            else:
                bias_gradient[-layer_idx] = delta
                weight_gradient[-layer_idx] = np.dot(delta, activations[-layer_idx-1].reshape(1,-1))
        #print("GOT TO THE END")
        return (bias_gradient, weight_gradient)

    def tune(self):
        # CONSIDER REMOVING THE TQDM ON EPOCHS
        hidden_node_vals = [1, 3, 5, 7, 9]
        epoch_vals = [10, 50, 100, 200, 500]
        momentum_vals = [0.5, 0.7, 0.9, 0.95, 0.99]
        learning_rate_vals = [0.0001, 0.001, 0.01, 0.1, 1.0]
        batch_size_vals = [16, 32, 64, 128, 256]

        hidden_node_scores = []
        epoch_scores = []
        momentum_scores = []
        learning_rate_scores = []
        batch_size_scores = []

        # Hidden node Count Tuning
        if (self.hidden_layer_count > 0):
            hidden_node_combinations = list(itertools.product(hidden_node_vals, repeat=self.hidden_layer_count))
            for combination in tqdm(hidden_node_combinations, desc="Tuning Hidden Node Count", leave=False):
                self.network_shape = [self.input_size] + (list(combination)) + ([self.class_count] if (self.prediction_type == "classification") else [1])
                hidden_node_score = self.train_test(tuning_flag=True, epochs=self.epochs, momentum=self.momentum, learning_rate=self.learning_rate, batch_size=self.batch_size)
                hidden_node_scores.append(np.mean(hidden_node_score))
            hidden_node_scores = np.array(hidden_node_scores)
            if self.prediction_type == "classification":
                self.network_shape = [self.input_size] + (list(hidden_node_combinations[np.argmax(hidden_node_scores)])) + ([self.class_count] if (self.prediction_type == "classification") else [1])
            else:
                self.network_shape = [self.input_size] + (list(hidden_node_combinations[np.argmin(hidden_node_scores)])) + ([self.class_count] if (self.prediction_type == "classification") else [1])          
            print(f"Tuned Network Shape: {self.network_shape}")

        # Epoch tuning
        for epochs in tqdm(epoch_vals, desc="Tuning Epochs", leave=False):
            epoch_score = self.train_test(tuning_flag=True, epochs=epochs, momentum=self.momentum, learning_rate=self.learning_rate, batch_size=self.batch_size)
            epoch_scores.append(np.mean(epoch_score))
        epoch_scores = np.array(epoch_scores)
        if self.prediction_type == "classification":
            self.epochs = epoch_vals[np.argmax(epoch_scores)]
        else:
            self.epochs = epoch_vals[np.argmin(epoch_scores)]
        print(f"Tuned Epoch Value: {self.epochs}")

        # Momentum Tuning
        for momentum in tqdm(momentum_vals, desc="Tuning Momentum", leave=False):
            momentum_score = self.train_test(tuning_flag=True, epochs=self.epochs, momentum=momentum, learning_rate=self.learning_rate, batch_size=self.batch_size)
            momentum_scores.append(np.mean(momentum_score))
        momentum_scores = np.array(momentum_scores)
        if self.prediction_type == "classification":
            self.momentum = momentum_vals[np.argmax(momentum_scores)]
        else:
            self.momentum = momentum_vals[np.argmin(momentum_scores)]
        print(f"Tuned Momentum Value: {self.momentum}")

        # Learning rate tuning
        for learning_rate in tqdm(learning_rate_vals, desc="Tuning Learning Rate", leave=False):
            learning_rate_score = self.train_test(tuning_flag=True, epochs=self.epochs, momentum=self.momentum, learning_rate=learning_rate, batch_size=self.batch_size)
            learning_rate_scores.append(np.mean(learning_rate_score))
        learning_rate_scores = np.array(learning_rate_scores)
        if self.prediction_type == "classification":
            self.learning_rate = learning_rate_vals[np.argmax(learning_rate_scores)]
        else:
            self.learning_rate = learning_rate_vals[np.argmin(learning_rate_scores)]
        print(f"Tuned Learning Rate: {self.learning_rate}")

        # Batch size tuning
        for batch_size in tqdm(batch_size_vals, desc="Tuning Batch Size", leave=False):
            batch_size_score = self.train_test(tuning_flag=True, epochs=self.epochs, momentum=self.momentum, learning_rate=self.learning_rate, batch_size=batch_size)
            batch_size_scores.append(np.mean(batch_size_score))
        batch_size_scores = np.array(batch_size_scores)
        if self.prediction_type == "classification":
            self.batch_size = batch_size_vals[np.argmax(batch_size_scores)]
        else:
            self.batch_size = batch_size_vals[np.argmin(batch_size_scores)]
        print(f"Tuned Batch Size: {self.batch_size}")

        return [self.network_shape, self.epochs, self.momentum, self.learning_rate, self.batch_size]
    
    def train_test(self, tuning_flag: bool, epochs=100, momentum=.9, learning_rate=.01, batch_size=10):
        scores = []
        if tuning_flag:
            for i in range(10):
                self.init_weights_biases_momentum()
                self.grad_desc(self.get_training_data(i), epochs, momentum, learning_rate, batch_size)
                score = self.loss(self.get_tuning_data())
                scores.append(score)
        else:
            for i in tqdm(range(10), desc="Evaluating Test Data", leave=False):
                self.init_weights_biases_momentum()
                self.grad_desc(self.get_training_data(i), self.epochs, self.momentum, self.learning_rate, self.batch_size)
                score = self.loss(self.get_testing_data(i))
                scores.append(score)
        return np.array(scores)
    
    def loss(self, test_data):
        if self.prediction_type == "classification":
            results = [(np.argmax(self.for_prop(example)), label) for (example, label) in test_data if not np.isnan(label)]
            correct_results = sum(int(example == label) for (example, label) in results)
            total_examples = len(results)
            return correct_results / total_examples
        else:
            results = [(self.for_prop(x), y) for (x, y) in test_data if not np.isnan(y)]
            # Ensure predictions and labels are both 1D arrays of the same length
            predictions = np.array([prediction.flatten()[0] if prediction.size == 1 else np.argmax(prediction) for (prediction, label) in results], dtype=float)
            labels = np.array([label for (prediction, label) in results], dtype=float)

            # Calculate MSE
            mse = np.mean((predictions - labels) ** 2)
            return mse
            

            '''
            results = [(self.for_prop(x), y) for (x, y) in test_data if not np.isnan(y)]
            predictions = np.array([prediction.flatten() for (prediction, label) in results], dtype=float).reshape(-1)
            labels = np.array([label for (prediction, label) in results], dtype=float).reshape(-1)
            mse = np.mean((predictions - labels) ** 2)
            return mse
            '''
        
    '''
    def evaluate(self, test_data):
        """Return the accuracy of the network on the test data, excluding any NaN labels."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data if not np.isnan(y) and len(x) == 2]
        correct_predictions = sum(int(x == y) for (x, y) in test_results)
        total_examples = len(test_results)
        return correct_predictions / total_examples if total_examples > 0 else 0
    '''

    '''
    def loss_prime(self):
        return
    '''
    def sigmoid(self, input: np):
        return 1.0/(1.0+np.exp(-input))
    def sigmoid_prime(self, input: np):
        return self.sigmoid(input)*(1-self.sigmoid(input))
    def softmax(self, input):
        exp = np.exp(input - np.max(input))
        return exp / np.sum(exp)
    '''
    # Since loss output is in a slightly different format for neural nets, we might need an final loss method to output final performance
    def final_loss(self):
        return
    '''
    # THIS PROLLY NEEDS EDITING
    def plot_loss(self, metrics: list, parameter: str, increment):
        '''
        This function plots the loss performance for each epoch. This allows us to visualize at how many epochs
        performance drops off.
        '''
        # Extract # of epochs and loss metrics
        metrics = np.array(metrics)
        epochs = np.arange(1, metrics.shape[0] + 1) * increment
        loss1 = metrics[:, 0]
        loss2 = metrics[:, 1]

        # Create loss plot
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, loss1, label='Loss Metric 1', marker='o')
        plt.plot(epochs, loss2, label='Loss Metric 2', marker='o')
        plt.xlabel(f'{parameter} Value')
        plt.ylabel('Loss')
        plt.title(f'Loss Metrics vs. {parameter} value')
        plt.legend()
        plt.grid(True)
        plt.show()
        plt.close()
    

# Neural Network Instantiation

In [31]:
abalone_data, cancer_data, fire_data, glass_data, machine_data, soybean_data = process_all('carlthedog3', True)

In [32]:
# Classification Sets
cancer_net_0 = neural_net(cancer_data, "classification", hidden_layer_count=0, epochs=10,momentum=.5,learning_rate=.01,batch_size=16)
cancer_net_1 = neural_net(cancer_data, "classification", hidden_layer_count=1, network_shape=[9,1,2],epochs=50,momentum=.7,learning_rate=.1,batch_size=16)
cancer_net_2 = neural_net(cancer_data, "classification", hidden_layer_count=2, network_shape=[9,1,5,2],epochs=50,momentum=.9,learning_rate=.01,batch_size=16)

glass_net_0 = neural_net(glass_data, "classification", hidden_layer_count=0, epochs=100,momentum=.5,learning_rate=.001,batch_size=16)
glass_net_1 = neural_net(glass_data, "classification", hidden_layer_count=1, network_shape=[9,1,6],epochs=500,momentum=.95,learning_rate=.01,batch_size=32)
glass_net_2 = neural_net(glass_data, "classification", hidden_layer_count=2, network_shape=[9,1,9,6],epochs=500,momentum=.95,learning_rate=.01,batch_size=32)

soybean_net_0 = neural_net(soybean_data, "classification", hidden_layer_count=0, epochs=10,momentum=.99,learning_rate=.01,batch_size=16)
soybean_net_1 = neural_net(soybean_data, "classification", hidden_layer_count=1, network_shape=[35,1,4],epochs=500,momentum=.99,learning_rate=.01,batch_size=16)
soybean_net_2 = neural_net(soybean_data, "classification", hidden_layer_count=2, network_shape=[35,1,9,4],epochs=500,momentum=.95,learning_rate=.1,batch_size=256)


# Regression Sets
abalone_net_0 = neural_net(abalone_data, "regression", hidden_layer_count=0, epochs=200,momentum=.9,learning_rate=.01,batch_size=32)
abalone_net_1 = neural_net(abalone_data, "regression", hidden_layer_count=1, network_shape=[8,1,1],epochs=500,momentum=.95,learning_rate=.01,batch_size=16)
abalone_net_2 = neural_net(abalone_data, "regression", hidden_layer_count=2, network_shape=[8,1,9,1],epochs=500,momentum=.99,learning_rate=.001,batch_size=16)

fire_net_0 = neural_net(fire_data, "regression", hidden_layer_count=0, epochs=200,momentum=.9,learning_rate=.01,batch_size=16)
fire_net_1 = neural_net(fire_data, "regression", hidden_layer_count=1, network_shape=[12,1,1],epochs=200,momentum=.95,learning_rate=.01,batch_size=16)
fire_net_2 = neural_net(fire_data, "regression", hidden_layer_count=2, network_shape=[12,1,1,1],epochs=500,momentum=.95,learning_rate=.001,batch_size=16)

machine_net_0 = neural_net(machine_data, "regression", hidden_layer_count=0, epochs=500,momentum=.95,learning_rate=.01,batch_size=16)
machine_net_1 = neural_net(machine_data, "regression", hidden_layer_count=1, network_shape=[9,1,1],epochs=500,momentum=.99,learning_rate=.01,batch_size=16)
machine_net_2 = neural_net(machine_data, "regression", hidden_layer_count=2, network_shape=[9,1,7,1],epochs=500,momentum=.99,learning_rate=.01,batch_size=16)

# Abalone Tuning + Final Scores

In [33]:
#abalone_net_0_parameters = abalone_net_0.tune()
#abalone_0_score = abalone_net_0.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(abalone_0_score)}")

In [34]:
#abalone_net_1_parameters = abalone_net_1.tune()
#abalone_1_score = abalone_net_1.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(abalone_1_score)}")

In [35]:
#abalone_net_2_parameters = abalone_net_2.tune()
#abalone_2_score = abalone_net_2.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(abalone_2_score)}")

# Fire Tuning + Final Scores

In [36]:
#fire_net_0_parameters = fire_net_0.tune()
#fire_0_score = fire_net_0.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(fire_0_score)}")

In [37]:
#fire_net_1_parameters = fire_net_1.tune()
#fire_1_score = fire_net_1.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(fire_1_score)}")

In [38]:
#fire_net_2_parameters = fire_net_2.tune()
#fire_2_score = fire_net_2.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(fire_2_score)}")

# Machine Tuning + Final Scores

In [39]:
#machine_net_0_parameters = machine_net_0.tune()
machine_0_score = machine_net_0.train_test(tuning_flag=False)
print(f"Average Performance: {np.mean(machine_0_score)}")

Average Performance: 0.0010063523995431142


In [40]:
#machine_net_1_parameters = machine_net_1.tune()
machine_1_score = machine_net_1.train_test(tuning_flag=False)
print(f"Average Performance: {np.mean(machine_1_score)}")

Average Performance: 0.0058771911516408756


In [41]:
#machine_net_2_parameters = machine_net_2.tune()
machine_2_score = machine_net_2.train_test(tuning_flag=False)
print(f"Average Performance: {np.mean(machine_2_score)}")

Average Performance: 0.009380829102971855


# Cancer Tuning + Final Scores

In [42]:
#cancer_net_0_parameters = cancer_net_0.tune()
cancer_0_score = cancer_net_0.train_test(tuning_flag=False)
print(f"Average Performance: {np.mean(cancer_0_score)}")

Average Performance: 0.47516641065028153


In [43]:
#cancer_net_1_parameters = cancer_net_1.tune()
cancer_1_score = cancer_net_1.train_test(tuning_flag=False)
print(f"Average Performance: {np.mean(cancer_1_score)}")

Average Performance: 0.9634664618535587


In [44]:
#cancer_net_2_parameters = cancer_net_2.tune()
cancer_2_score = cancer_net_2.train_test(tuning_flag=False)
print(f"Average Performance: {np.mean(cancer_2_score)}")

Average Performance: 0.9650025601638503


# Glass Tuning + Final Scores

In [45]:
#glass_net_0_parameters = glass_net_0.tune()
#glass_0_score = glass_net_0.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(glass_0_score)}")

In [46]:
#glass_net_1_parameters = glass_net_1.tune()
#glass_1_score = glass_net_1.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(glass_1_score)}")

In [47]:
#glass_net_2_parameters = glass_net_2.tune()
#glass_2_score = glass_net_2.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(glass_2_score)}")

# Soybean Tuning + Final Scores

In [48]:
#soybean_net_0_parameters = soybean_net_0.tune()
#soybean_0_score = soybean_net_0.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(soybean_0_score)}")

In [49]:
#soybean_net_1_parameters = soybean_net_1.tune()
#soybean_1_score = soybean_net_1.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(soybean_1_score)}")

In [50]:
#soybean_net_2_parameters = soybean_net_2.tune()
#soybean_2_score = soybean_net_2.train_test(tuning_flag=False)
#print(f"Average Performance: {np.mean(soybean_2_score)}")

# Video Demo
## Performance From One Test Fold

In [51]:
# Task 1
print("Classification Network Performance on Fold 0 (Cancer)")
print(f"0 Hidden Layers Performance: {cancer_0_score[0]}")
print(f"1 Hidden Layers Performance: {cancer_1_score[0]}")
print(f"2 Hidden Layers Performance: {cancer_2_score[0]}")
print()
print("Regression Network Performance on Fold 0 (Machine)")
print(f"0 Hidden Layers Performance: {machine_0_score[0]}")
print(f"1 Hidden Layers Performance: {machine_1_score[0]}")
print(f"2 Hidden Layers Performance: {machine_2_score[0]}")

Classification Network Performance on Fold 0 (Cancer)
0 Hidden Layers Performance: 0.3333333333333333
1 Hidden Layers Performance: 0.9682539682539683
2 Hidden Layers Performance: 0.9841269841269841

Regression Network Performance on Fold 0 (Machine)
0 Hidden Layers Performance: 0.0001844605363055439
1 Hidden Layers Performance: 0.00011312847234649855
2 Hidden Layers Performance: 0.0002376569890918629


## Sample Model

In [52]:
# Task 2
print(f"Cancer w/ 1 Hidden Layer Network Shape: {cancer_net_1.network_shape}")
print(f"Weights Between Input Layer and Hidden Layer:\n{cancer_net_1.weights[0]}")
print(f"Weights Between Hidden Layer and Output Layer:\n{cancer_net_1.weights[1]}")
print()
print(f"Machine w/ 1 Hidden Layer Network Shape: {machine_net_1.network_shape}")
print(f"Weights Between Input Layer and Hidden Layer:\n{machine_net_1.weights[0]}")
print(f"Weights Between Hidden Layer and Output Layer:\n{machine_net_1.weights[1]}")

Cancer w/ 1 Hidden Layer Network Shape: [9, 1, 2]
Weights Between Input Layer and Hidden Layer:
[[-2.21471707 -0.68837427 -1.13672872 -1.20929951 -0.0200594  -3.0890679
  -1.74855564 -0.99468013 -1.54334363]]
Weights Between Hidden Layer and Output Layer:
[[ 5.53396016]
 [-4.9073779 ]]

Machine w/ 1 Hidden Layer Network Shape: [9, 1, 1]
Weights Between Input Layer and Hidden Layer:
[[-0.22329062 -0.96813138 -0.29822035  0.98167322  1.54457181  0.7522363
   0.48431006  0.3816307   1.49163571]]
Weights Between Hidden Layer and Output Layer:
[[1.15765069]]


## Example Propogated Through a Two-Layer Network

In [53]:
# Task 3
test_example = np.array([1,2,3,4,5,6,7,8,9])
print(f"Features:\n{test_example}")
print()
print(f"Weights:\n{cancer_net_2.weights}")
print()
print(f"Biases:\n{cancer_net_2.biases}")
print()
print(f"Network Shape: {cancer_net_2.network_shape}")
print()
cancer_net_2.for_prop(test_example, testing=True)

Features:
[1 2 3 4 5 6 7 8 9]

Weights:
[array([[-0.11669517,  0.94290991,  0.644627  ,  2.35457545,  0.59501385,
         2.46126651,  1.12322376,  2.57345388, -1.4759378 ]]), array([[-0.69698908],
       [-2.57937776],
       [ 1.0149423 ],
       [ 2.47991119],
       [-2.02879198]]), array([[ 0.55321781,  2.50209572, -0.39936093, -3.19035069,  0.50505143],
       [-0.30237229, -1.48813122,  1.0438426 ,  2.22667784, -2.31520208]])]

Biases:
[array([[-2.2248623]]), array([[-0.42149764],
       [ 0.98206794],
       [ 0.81549606],
       [-0.78616921],
       [ 0.58411359]]), array([[ 0.70700872],
       [-1.23519172]])]

Network Shape: [9, 1, 5, 2]

Activation at Layer 1:
[[1.]]

Activation at Layer 2:
[[0.24629209]
 [0.16835794]
 [0.86181394]
 [0.84471563]
 [0.19082192]]

Activation at Output Layer:
[[0.07894681]
 [0.92105319]]



array([[0.07894681],
       [0.92105319]])

## Gradient Calculation

In [54]:
# Task 4
test_example = np.array([1,2,3,4,5,6,7,8,9])
cancer_net_0.epoch(test_example, 1, testing=True)
machine_net_0.epoch(test_example, 1, testing=True)

Activations:
[[2.60669553e-06]
 [9.99997393e-01]]
Label:
[[0]
 [1]]

Gradient at Output Layer (delta):
[[ 2.60669553e-06]
 [-2.60669553e-06]]

Activations:
[[6.97216574]]
Label:
1

Gradient at Output Layer (delta):
[[5.97216574]]



([array([[5.97216574]])],
 [array([[ 5.97216574, 11.94433149, 17.91649723, 23.88866298, 29.86082872,
          35.83299446, 41.80516021, 47.77732595, 53.7494917 ]])])

## Weight Updates on a Two-Layer Network

In [55]:
# Task 5
test_batch = [(np.array([1,2,3,4,5,6,7,8,9]),1)]
print(f"Classification Weight Updates\n")
cancer_net_2.update_weights(test_batch,.9,.01,testing=True)
print()
print(f"Regression Weight Updates\n")
machine_net_2.update_weights(test_batch,.99,.01,testing=True)

Classification Weight Updates

Weight update at Layer 1:[[-0.11584351  0.94340389  0.64509927  2.35424046  0.59467703  2.46137942
   1.12325131  2.57326038 -1.47544261]]
Weight update at Layer 2:[[-0.69714671]
 [-2.57986224]
 [ 1.01509275]
 [ 2.4805032 ]
 [-2.02917982]]
Weight update at Layer 3:[[ 0.55348427  2.5028798  -0.39932881 -3.19079922  0.50569449]
 [-0.30263875 -1.4889153   1.04381048  2.22712637 -2.31584514]]

Regression Weight Updates

Weight update at Layer 1:[[-1.86727577  0.46261783  0.21700525  0.26026253  2.45769752  0.20109865
   0.65733924  2.73641307  0.3090524 ]]
Weight update at Layer 2:[[-3.56959242]
 [-5.24217885]
 [-1.8243206 ]
 [-5.00511405]
 [-3.47656587]
 [-3.03899549]
 [-6.30863235]]
Weight update at Layer 3:[[-8.28306981  0.65104018 -2.27090771 -3.31039135  1.10192273  3.63388279
   2.00065752]]


## Average Performance for 0,1, and 2 Hidden Layers on Cancer and Machine Datasets

In [56]:
# Task 6
print(f"Cancer Net w/ 0 Hidden Layers Average Performance: {np.mean(cancer_0_score)}")
print(f"Cancer Net w/ 1 Hidden Layers Average Performance: {np.mean(cancer_1_score)}")
print(f"Cancer Net w/ 2 Hidden Layers Average Performance: {np.mean(cancer_2_score)}")
print()
print(f"Machine Net w/ 0 Hidden Layers Average Performance: {np.mean(machine_0_score)}")
print(f"Machine Net w/ 1 Hidden Layers Average Performance: {np.mean(machine_1_score)}")
print(f"Machine Net w/ 2 Hidden Layers Average Performance: {np.mean(machine_2_score)}")

Cancer Net w/ 0 Hidden Layers Average Performance: 0.47516641065028153
Cancer Net w/ 1 Hidden Layers Average Performance: 0.9634664618535587
Cancer Net w/ 2 Hidden Layers Average Performance: 0.9650025601638503

Machine Net w/ 0 Hidden Layers Average Performance: 0.0010063523995431142
Machine Net w/ 1 Hidden Layers Average Performance: 0.0058771911516408756
Machine Net w/ 2 Hidden Layers Average Performance: 0.009380829102971855
